# HW8 Problems
### Learning objectives
* Gain experience in using the Particle Data Group compilations of decay rates to determine relative branching fractions for different decay modes and to develop insight on decay kinematics. 
* Using the example of $D_s^{+}\rightarrow K^+K^-\pi^{+}$, explore how Dalitz Plots can  be  used to understand resonant structure in hadron decays.

## Question 1:

B-factories are a good place to study the decays of bottom hadrons.  These $e^+e^-$ colliders typically operate at a center-of-mass energy of 10.579 GeV, the mass of the $\Upsilon (4s)$.  This $b\overline b $ resonance is just above the open bottom threshold and therefore decays (more than 96% of the time) to
a $B\overline B$ (there is not enough additional energy to produce even a single additional pion).

Consider the case $\Upsilon (4s)\rightarrow B^0\overline{B^0}$.  If the $\overline{B^0}$ is fully reconstructed, then we know that all the remaining particles must come from the decay of the $B^0$ and if the $B^0$ is fully reconstructed, then we know all the remaining particles must come from the deacy of the $\overline{B^0}$. This strategy can therefore be used to measure absolute branching fractions.


### 1a (2 pts)

Consider the production $e^+e^- \rightarrow  B\overline B$ followed by the
decay chain for the $B^0$ and $\overline{B^0}$.  Some fraction of the time, the $B^0$ will decay through the following decay chain:
\begin{eqnarray*}
B^0 & \rightarrow & D_s^{*+}\pi^-\\
D_s^{*+} & \rightarrow & D_s^+ \gamma\\
D_s^{+}  & \rightarrow & K^+K^- \pi^+
\end{eqnarray*}

Here the $D_s^{+}$ decay to $K^+K^-\pi^+$ can occur through several mechanisms, including:

\begin{eqnarray*}
D_s^{+} \rightarrow \phi \pi^{+} & \;\;{\rm followed\; by} & \; \phi\rightarrow K^{+} K^{-}\\
D_s^{+} \rightarrow  K^{*0} K^{+} & \;\;{\rm followed\; by} & K^{*0}\rightarrow K^{-} \pi^{+} {\rm or\ } K^{*0}\rightarrow K^{+} \pi^{-}\\ 
D_s^{+}  \rightarrow  K^+K^- \pi^+ & & {\rm three-body\; (non-resonant)}
\end{eqnarray*}

For each of the particles listed below, use the Particle Data Group (PDG) tables to find their mass and spin. (Note, each of the particles listed above will have an anti-particle with opposite charge and the same mass and spin as the particle.  The $\overline{B^0}$ can decay through a decay chain where all the particles above are replaced by their anti-particles). Please note that as there are two $B$-mesons in each event, you will need to consider decays of both the $B$ mesons to find all the final state particles.

| Particle | Mass | Spin |
| --- | --- | --- |
| $B^0$ |  | |
| $D_s^{*+}$ |   |  |
| $D_s^+$ | |  |
| $\phi$ |  |  |
| $K^{*+}$ |  |  |
| $K^+ $ |  |  |
| $\pi^+ $ |  |  |
| $\gamma $ |  |  |
| $D^{*+} $ | |  |
| $D^+ $ | | |
| $D^0 $ | | |



### 1b (2 pts)

The $D_s^{*+}$ is composed of a charm quark and an anti-strange quark.  There is a similar state consisting of a charm quark and an anti-down quark called the $D^{*+}$.
Use the results tabulated by the PDG to compare the branching fractions for  $D_s^{*+}\rightarrow D_s^+ \gamma$ and $D^{*+}\rightarrow D^+ \gamma$.  From the properties listed in the PDG, can you explain why are these so different?

Your answer here.

### 1c (5 pts)

As noted above,  $D_s\rightarrow K^+K^-\pi$ decay can occur through several possible intermediate resonance states.  In principle the decays to these should be combined by adding their decay amplitudes and then squaring the sum, since they can quantum mechanically interfere.  However, the resonant structure is still evident in the decay kinematics and it is possible to observe this resonant structure using a Dalitz plot. The file DsstarPiMC.dat contains a  simulated sample that corrresponds to the decay chain described above.

Some of our events contain a decay chain starting with $B^{0} \rightarrow  D_s^{*+}\pi^{-}$
where the $D_S^{*+}$ decays to a $D_s^+ \rightarrow K^+ K^- \pi^+$.  Other events contain a decay chain starting with $\overline{B^0} \rightarrow  D_s^{*-}\pi^+$
where the $D_S^{*-}$ decays to a $D_s^{+} \rightarrow K^{+} K^{-} \pi^{-}$.  In both these cases, the
final state particles are $K^{+}K^{-}\pi^{+}\pi^{-}$.  By selecting the $K^{+}K^{-}\pi^{\pm}$ combination with
invariant mass consistent with the $D_s^{\pm}$,  determine for each event which $\pi$ comes from the $D_s^\pm$ decay.  Using the $\pi^\pm$ you have selected, plot the invariant mass of the $K^\mp \pi^\pm$ combination in the region around the $K^{*0}$.  Do you see a peak in roughly the right place?  

In [7]:
import math
import numpy as np
import matplotlib.pyplot as plt

# Parse the input file.
file = "DsstarPiMC.dat"

# Information needed to interpret the input file structure
# The order of variables is explained in the metadata at the top of the file
pMc = 0              # Magnitude of (three)-momentum of the particle in the lab frame
massMc=1             # mass of the particle
thetaMc=2            # polar angle of the particle
phiMc=3              # azimuthal angle of the  particle
idMc=4               # PDG ID of the particle (See http://www-pdg.lbl.gov/2018/reviews/rpp2018-rev-monte-carlo-numbering.pdf) 
mothMc=5             # index to the mother of the particle (the row corresponding to the mother, where
                     # the numbering of rows for each event starts at zero
nDauMc=6             # number of daughters the particle has
#
# The pdg IDs of the particles we care about
pdgIDKplus=0 # update this value
pdgIDKminus=0 # update this value
pdgIDPiplus=0 # update this value
pdgIDPiminus=0 # update this value
DsMass = 0 # in GeV # update this value

# This loop will process the file
inEvent = False
events = []
for line in open(file,"r"):
    line = line.strip()
    info = line.split(",")    
    if info[0] =="<event>":
        inEvent = True
        particles = []
    elif inEvent and (info[0] == "</event>"):
        events.append(particles)
        inEvent = False
    elif inEvent:
        b = [float(info[i]) for i in range(7)]
        particles.append(b)
numEv = len(events)

# update this function
def makeFourVector(momentum_magnitude, mass, theta, phi):
    """Make a four momentum vector object given the particle kinematic info
    
    Parameters
    ==========
    momentum_magnitude : float
      magnitude of the three momentum
      
    mass : float
      particle mass
      
    theta : float
      polar angle of particle momentum vector
      
    phi : float
      azimuthal angle of particle vector
      
    Returns
    =======
    fv : array
      a 4-element numpy array where component 0 is the time-like component of
      the particle's four momentum and components 1-3 are the three momentum
    """
    
    '''Your code here'''
    return fv

#update this function
def invMsq(a):
    """Compute the square of the invariant mass from a four momentum vector
    
    Parameters
    ==========
    a : array
      a four-element array representing the four momentum vector of a particle
      
    Returns
    =======
    v : float
      the squared invariant mass of the particle 
    """
    
    '''Your code here'''
    return v

# Arrays to store the invariant mass squared of 2-particle pairs
MsqKK = np.zeros(numEv)
MsqKpi = np.zeros(numEv)

# Arrays to store the mass of 2-particle pairs
MassKK = np.zeros(numEv)
MassKPi = np.zeros(numEv)

# Objects to store the four-momenta of the K+, K-,  pi+ and pi-
K1=np.zeros([numEv,4])
K2=np.zeros([numEv,4])
Pi1=np.zeros([numEv,4])

nDplus=0
nDminus=0

# This is a loop over the simulated events, each of which contains a list
# of particles.
for i in range(numEv):
    '''Your code here'''

# Plot the M(K pi) histogram
'''Your code here'''

'Your code here'

### 1d (2 pts)
Plot the invariant mass of the the two kaons, $K^+$ and $K^-$ system. You will want to adjust the range of the $x$ axis to focus on the region around the $\phi$ mass.  Do you see the $\phi$ and is its mass roughly in the right place? (You will fit for this mass later in the problem)

In [6]:
'''Your code here'''

'Your code here'

### 1e (2 pts)
Make the Dalitz plot for variables $m_{KK}^2$ and $m_{\pi K}^2$ (combining the kaon and the pion of the opposite charge).

In [5]:
# Plot the results as a scatter plot
'''Your code here'''

### 1f (2 pts)

Explain the structure you see in the Dalitz plot you have created.

Your answer here


### 1g (5 pts)
Using these data, compute the branching fractions for decays
\begin{eqnarray*}
D_s^+ &\rightarrow &  \phi \pi^+\\
D_s^+ & \rightarrow &  \overline K^{*0}  K^+
\end{eqnarray*}
as well as the nonresonant
$$                                                                                            
D_s^+ \rightarrow K^+ K^- \pi^+                                                                
$$
and compare them to the values given in the PDG.

Note: The shape of a resonance decay is a Breit-Wigner.  The non-resonant background shape is non-trivial to predict, but if you only look in the region close to the resonant mass, approximating the background using a linear function does a reasonable job. 

In [4]:
from scipy.optimize import curve_fit

# We are going to fit to a relativistic Breit-Wigner for the signal and a linaer function for the background
def fit_relBWplusBckgnd(x, N, Mass, Gamma, A, B):
    """Defines a relativistic Breit-Wigner function added to a linear background
    
    Parameters
    ==========
    x : float
      input energy to the function
      
    N : float
      scaling factor
      
    Mass : float
      central resonance of the distribution
      
    Gamma : float
      width of the resonance
      
    A : float
      intercept of linear background distribution
      
    B : float
      slope of linear background distribution
      
    Returns
    =======
    BW : float
      the value of the BW+linear background function at the input energy, x
    """
    '''Your code here'''
    return A+B*x+N*k/((x**2-Mass**2)**2+(Mass*Gamma)**2)

nBins=0

# First fit for the K* peak
minMass=0
maxMass=0

#Plot the data
'''Your code here'''

# Fit the data
'''Your code here'''

# Add the fit function as a curve
'''Your code here'''

# Store fitted number of K* 
numKstar = 0

# Now do the same for the Phi Peak
minMass=0
maxMass=0

#Plot the data
'''Your code here'''

# Fit the data
'''Your code here'''

# Add the fit function as a curve
'''Your code here'''

# Add the fit function as a curve

# Store fitted number of Phi
numPhi = 0

# Calculate the number of non-resonant (what is left)

Your answer here